<a href="https://colab.research.google.com/github/tahreemrasul/gemini_workshops/blob/main/simple_agent_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build an agent from scratch using Gemini

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/tahreemrasul/gemini_workshops/blob/main/simple_agent_from_scratch.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/tahreemrasul/gemini_workshops/blob/main/simple_agent_from_scratch.ipynb">
      <img width="32px" src="https://raw.githubusercontent.com/primer/octicons/refs/heads/main/icons/mark-github-24.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>    

| Author |
| --- |
| [Tahreem Rasul](https://github.com/tahreemrasul/) |

This notebook shows how to build a simple agent using the Gemini API and the
[Google GenAI Python SDK](https://pypi.org/project/google-genai/).

At its core, an agent is just:

1. An LLM (Gemini).
2. A set of tools (Python functions the model can call).
3. A loop that lets the model **think → act → observe → continue thinking**.

We’ll:

- Define a tiny `get_weather` tool.
- Let Gemini decide when to call it (via function calling).
- Wrap that into a minimal `while True` loop to make it *agentic*.

## Getting Started

### Install & Import Dependencies

In [ ]:
!pip install -U -q google-genai

In [ ]:
import os
import getpass
from typing import Dict

from google import genai
from google.genai import types

print("google-genai version:", genai.__version__)

### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the following cell to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Get API Key

There are a few different ways of accessing the Gemini models in your applications. Most notably, you can do this through Vertex AI in Google Cloud Platform, or via the API method using Google AI Studio. We will be using the second approach.

Head over to https://aistudio.google.com/ and generate an API Key.
1. Click on **API Keys** on the left sidebar.
2. You should now see an option to **Create API Key**. Click on this.
3. A dialog box will open. You will be prompted to enter the name of the key. Enter a name of your choice. You also need to link this to a Google Cloud Project. From the dropdown of project selection, select **Create project**. You will be prompted to name the project, choose a name of your choice. Click on **Create Key**.
4. You should now see your created API key in the list. Click on the key and copy it. Paste it in the cell below.


In [ ]:
# 🔑 Paste your API key from Google AI Studio
API_KEY = "your-api-key"

from google import genai
client = genai.Client(api_key=API_KEY)

### Choose a model

In [ ]:
MODEL_ID = "gemini-2.5-flash"  # @param {type: "string"}

## Build the agent

### Step 1 - Define a simple weather tool


We'll start with a mock weather function:

- In a real app, this would call a live weather API.
- In this notebook, it just returns hard-coded values for a few cities.

Then we describe it to Gemini using a **FunctionDeclaration** + **Tool**, so the
model knows:

- the tool name (`get_weather`)
- what it does
- what parameters it expects (`city: string`)

In [ ]:
def get_weather_impl(city: str) -> Dict:
    """Mock weather function.

    In a real app, this would call an external API.
    Here we just return fake but structured data.
    """
    fake_temps = {
        "berlin": {"temperature_c": 17, "condition": "cloudy"},
        "london": {"temperature_c": 15, "condition": "windy"},
        "lahore": {"temperature_c": 32, "condition": "hot"},
        "islamabad": {"temperature_c": 28, "condition": "warm and clear"},
        "paris": {"temperature_c": 19, "condition": "light rain"},
    }

    city_key = city.lower()
    # return default fallback
    data = fake_temps.get(
        city_key,
        {"temperature_c": 22, "condition": "mild and unknown"},
    )
    return {
        "city": city,
        "temperature_c": data["temperature_c"],
        "condition": data["condition"],
    }

We will now describe the function to the model.

In [ ]:
get_weather_decl = types.FunctionDeclaration(
    name="get_weather",
    description="Get the current (mock) weather for a given city.",
    parameters={
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "City name, e.g. 'London' or 'Lahore'.",
            }
        },
        "required": ["city"],
    },
)

The final step would be to wrap this function as a tool.

In [ ]:
weather_tool = types.Tool(function_declarations=[get_weather_decl])

print(weather_tool)

### Step 2 — Make a single call with tools

Before we build a loop, let's:

1. Send a user query.
2. Give Gemini our `weather_tool`.
3. Inspect whether the model **requests a function call**.

We'll deliberately not let the SDK auto-handle function calls; instead,
we'll inspect `response.function_calls` ourselves so the agent loop is explicit.

In [ ]:
def call_model_once(user_input: str):
    """Send one message to the model with our weather tool attached."""
    response = client.models.generate_content(
        model=MODEL_ID,
        contents=[user_input],
        config=types.GenerateContentConfig(
            tools=[weather_tool],
            # optional: temperature=0.2, etc.
        ),
    )
    return response

In [ ]:
test_response = call_model_once("What's the weather like in Islamabad right now?")
print("\nFunction calls:", test_response.function_calls)

### Step 3 — Build the minimal agent loop

Now we'll create a loop:

1. Read user input.
2. Ask the model what to do (with tools available).
3. **If** the model asks to call `get_weather`, we:
   - parse the tool arguments,
   - run `get_weather_impl(...)`,
   - send the result back to the model as a *tool response*.
4. Print the model's final answer.
5. Repeat.

This is the essence of an agent:

> **reason → act (tool) → observe (tool result) → continue reasoning**


In [ ]:
def run_minimal_agent():
    print("Gemini agent. Type 'exit' or 'quit' to stop.\n")

    while True:
        user_input = input("You: ")
        if user_input.strip().lower() in {"exit", "quit", "q"}:
            print("👋 Bye!")
            break

        # 1) First turn: ask the model what to do, with tools available
        response = client.models.generate_content(
            model=MODEL_ID,
            contents=[user_input],
            config=types.GenerateContentConfig(
                tools=[weather_tool],
            ),
        )

        # 2) If the model decided to call a tool, handle it
        if response.function_calls:
            tool_call = response.function_calls[0]
            print(f"[debug] Model wants to call: {tool_call.name} with args {tool_call.args}")

            if tool_call.name == "get_weather":
                # Extract arguments from the function call
                city = tool_call.args.get("city", "")
                tool_result = get_weather_impl(city=city)

                # Wrap the tool result as a function response part
                function_response_part = types.Part.from_function_response(
                    name=tool_call.name,
                    response=tool_result,
                )

                # 3) Second turn: send user + tool result back to the model
                response = client.models.generate_content(
                    model=MODEL_ID,
                    contents=[
                        user_input,              # original user message
                        function_response_part,  # tool response
                    ],
                    config=types.GenerateContentConfig(
                        tools=[weather_tool],
                    ),
                )

        # 4) Print the final answer
        print("Agent:", response.text)
        print("-" * 60)

run_minimal_agent()

### Step 4 — Play with it: make it feel more "agentic"

Some ideas to extend this notebook:

1. **Add a second tool**  
   For example: `convert_temperature` that converts °C ↔ °F.
   - Declare it as another `FunctionDeclaration`.
   - Add it to the same `Tool` or another tool in `GenerateContentConfig`.
   - Update the loop to handle calls to either `get_weather` or `convert_temperature`.

2. **Add simple memory**  
   Keep a `history` list of previous turns:
   ```python
   history = []
   # each loop:
   history.append({"role": "user", "content": user_input})
   history.append({"role": "agent", "content": response.text})
